In [1]:
import bs4 as bs4
import urllib.request
import requests
import time
import re

site = 'https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=1'


source = urllib.request.urlopen(site).read()
time.sleep(5)

soup = bs4.BeautifulSoup(source,'html')

In [2]:
cards_imovel = soup.find_all('div', class_='slide col-xs-12 col-sm-6 col-md-6 col-lg-4')

In [24]:
teste_infos_add = [card.find_all('span', class_='tooltip-control') for card in cards_imovel]

In [59]:
len(teste_infos_add)

9

In [57]:
dict_imoveis = {}

# for i in range(0, len(teste_infos_add)) as j:
# itter = []
for i in range(0, ):
    primeiro_texto = teste_infos_add[0][i].img
    print(primeiro_texto)
    """
    Procurando por metragem quadrada no anuncio do imovel
    """
    if re.findall(r"esquadro.png", str(primeiro_texto)):
        print("encontrei!")
        
    else:
        print('f')
        

<img src="https://itaivan.com/wp-content/uploads/2020/04/esquadro.png"/>
encontrei!
<img src="https://itaivan.com/wp-content/uploads/2020/04/cama.png"/>
f
<img src="https://itaivan.com/wp-content/uploads/2020/04/galpao.png"/>
f


IndexError: list index out of range

In [52]:
primeiro_texto = teste_infos_add[0][0].img
resultado = re.findall(r"esquadro.png", str(primeiro_texto), )
resultado

['esquadro.png']

In [60]:
teste_imagem = [card.find('div', class_='ls-picture').get('style') for card in cards_imovel]
tipo_imovel = [card.find('div', class_='ls-categoria').get_text().strip() for card in cards_imovel]
local_imovel = [card.find('span', class_='ls-loc block').get_text().strip().replace(r' ','') for card in cards_imovel]
codigo_imovel = [card.find('span', class_='ls-cod').get_text() for card in cards_imovel]
m2_imovel = [card.find('span', class_='tooltip-control').get_text().strip() for card in cards_imovel]
preco_imovel = [card.find('span', class_='block ls-price color').get_text().strip() for card in cards_imovel]

#local_imovel
# teste_imagem
# tipo_imovel
#imagem[21:]

In [103]:
def data_cleaning(card_iterator=local_imovel,
                  codigo_imovel = codigo_imovel):
    
    bairro = []
    cidade = []
    
    cod_imoveis = []
    
    for i in range(len(local_imovel)):
        bairro.append(local_imovel[i].split()[0])
        cidade.append(local_imovel[i].split()[1])
    
    for i in range(len(codigo_imovel)):
        cod_imoveis.append(codigo_imovel[i].split()[1])
    
    return bairro, cidade, cod_imoveis

In [102]:
#re.compile(r'[/d]+', codigo_imovel[0])
codigo_imovel[1].split()

['Cód:', '13155']

In [104]:
bairro, cidade, cod_imoveis = data_cleaning()

In [105]:
cod_imoveis

['74610', '13155', '74608', '4176', '4172', '74605', '13146', '13145', '13150']

In [106]:
import pandas as pd

data = pd.DataFrame(list(zip(tipo_imovel, bairro, cidade, 
                              cod_imoveis, m2_imovel,
                             preco_imovel)), columns=
                    ['Tipo_imovel', 'bairro', 'cidade','codigo_imovel',
                     'm2_imovel','preco_imovel'])
data.head()

,Tipo_imovel,bairro,cidade,codigo_imovel,m2_imovel,preco_imovel
0,Apartamento,Centro,JaraguáDoSul,74610,"437,64 m²","R$ 3.000.000,00"
1,Casa,VilaNova,JaraguáDoSul,13155,"112,53 m²","R$ 1.380.000,00"
2,Apartamento,Amizade,JaraguáDoSul,74608,2,"R$ 170.000,00"
3,Terreno,VilaChartres,JaraguaDoSul,4176,"511,40 m²","R$ 437.000,00"
4,Terreno,RioDaLuz,JaraguáDoSul,4172,"7.561,40 m²","R$ 1.510.000,00"


In [84]:
data.to_excel('teste_scraping1.xlsx')

In [62]:
centro = data['Local_imovel'].strip()

AttributeError: 'Series' object has no attribute 'strip'

In [10]:
import requests
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

url = 'https://www.melhorcambio.com/dolar-hoje'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

response = requests.get(url, headers=headers).text

soup = BeautifulSoup(response, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Dólar Hoje: Cotação comercial e turismo. Valor e Preço!
  </title>
  <meta content="Dólar Hoje. Só aqui você confere a cotação Turismo do Dólar dos Estados Unidos, não só a comercial! Calcule e compare onde comprar melhor num só lugar!" name="description"/>
  <meta content="https://www.melhorcambio.com/img/icone-mc.jpg" property="og:image">
   <meta content="website" property="og:type"/>
   <link href="https://www.melhorcambio.com/dolar-hoje" rel="canonical"/>
   <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
   <link href="//fonts.googleapis.com/css?family=Montserrat:400,700" rel="stylesheet" type="text/css"/>
   <link href="/dolar_hoje/css/foundation.css" rel="stylesheet"/>
   <link href="/dolar_hoje/css/animate.css" rel="stylesheet"/>
   <link href="/dolar_hoje/css/site.css?v=582a380" rel="stylesh

In [32]:
tabelas = soup.find('div', {"id":"snipet-div"})

teste = [soup.find('td', class_='tdvalor').get_text() for tabela in tabelas]
print(teste)

['R$5,57', 'R$5,57', 'R$5,57', 'R$5,57']


In [168]:
data.shape

(9, 5)

In [5]:
len(teste)

9

In [ ]:
           # Base da pesquisa!
cards_usuario = soup.find_all('div', class_='user-list-item py-4 d-flex hx_hit-user')
            
            teste = [card.find('p', class_='mb-1') for card in cards_usuario] 
            descricao_usuario = ['Nao_informado' if v is None else v.get_text() for v in teste]
            
            teste_local = [card.find('div', class_='mr-3') for card in cards_usuario]
            local_usuario = ['Nao_informado' if v is None else v.get_text().strip() for v in teste_local]
            
            teste_nome = [card.find('a', class_='mr-1')['href'] for card in cards_usuario]
            nome_usuario = ['Nao_informado' if v is None else v for v in teste_nome]
            
            data = pd.DataFrame(list(zip(nome_usuario, local_usuario,descricao_usuario)),
                   columns=['Nome_usuario', 'Local_usuario', 'Descricao_usuario'])
            